In [3]:
import os

from IPython.core import magic_arguments
from IPython.core.getipython import get_ipython
from IPython.core.magic import (Magics, magics_class,  cell_magic)          

In [10]:
@magics_class
class IPhoneNotification(Magics):
    
    CMD = """osascript<<END

    tell application "Messages"

    send "{body}" to buddy "{phone}" of (service 1 whose service type is iMessage)

    end tell

    END"""
    
    @magic_arguments.magic_arguments()
    @magic_arguments.argument('-p', '--phone', type=str,
        help="""Phone number where to send text message""")
    @magic_arguments.argument('-m', '--message', type=str, default="job complete",
        help="""text message to send""")
    
    @cell_magic
    def phone_notification(self, line, cell):
        args = magic_arguments.parse_argstring(self.phone_notification, line)
        
        try:
            x = self.shell.run_cell(cell)           
        finally:
            try:
                # remove quotes
                message = args.message.replace('"', "")
                
                # if failed then try to include error message
                if not x.success:
                    print x
                    print dir(x)
                    body = "[Failed] {} \n Error: {}".format(message, x.error_in_exec or "Unknown")
                # if successful 
                else:
                    body = "[Success] {}".format(message)
                
                # use apple script to send message
                cmd = self.CMD.format(body=body, phone=args.phone)          
                os.system(cmd)
            except Exception, ex:
                print cmd
                raise
                
get_ipython().register_magics(IPhoneNotification)      